In [2]:
import geopandas as gpd

# Load the GeoJSON file into a GeoDataFrame
def load_geojson_to_dataframe(file_path):
    """
    Load a GeoJSON file into a GeoDataFrame.

    :param file_path: Path to the GeoJSON file.
    :return: A GeoDataFrame containing the GeoJSON data.
    """
    try:
        gdf = gpd.read_file(file_path)
        print(gdf.head())  # Display the first few rows
        return gdf
    except Exception as e:
        print(f"Error loading GeoJSON file: {e}")
        return None

# Example usage
geojson_file = "file_backup/OSM-data-20241230.geojson"  # Replace with your GeoJSON file path
geo_df = load_geojson_to_dataframe(geojson_file)

# Convert to a pandas DataFrame (if geometry is not needed)
if geo_df is not None:
    df = geo_df.drop(columns="geometry")  # Drop the geometry column
    print(df.head())


                id              @id Latitude Longitude access addr:city  \
0   node/356723845   node/356723845     None      None   None      None   
1   node/356736649   node/356736649     None      None   None      None   
2   node/356736651   node/356736651     None      None   None      None   
3   node/356852153   node/356852153     None      None   None      None   
4  node/1655546236  node/1655546236     None      None   None      None   

  addr:county addr:full alt_name check_date  ... subject:wikipedia support  \
0        None      None     None       None  ...              None    pole   
1        None      None     None       None  ...              None    None   
2        None      None     None       None  ...              None    None   
3        None      None     None       None  ...              None    None   
4        None      None     None       None  ...              None    None   

     thc:designation thc:inscription_date  \
0  Historical Marker               

In [6]:
from hmdb import write2csv

write2csv(df,"OSM-data",True)


./file_backup/20241230_OSM-data written to disk.


In [ ]:
import overpy

# Initialize the Overpass API
api = overpy.Overpass()

def get_node_by_id(node_id):
    """
    Search for a node in OSM by its ID using the Overpass API.
    
    :param node_id: The ID of the node to search for.
    :return: Node details (latitude, longitude, and tags) if found, else None.
    """
    try:
        # Query Overpass API for the node by its ID
        query = f"node({node_id}); out;"
        result = api.query(query)
        
        # Check if the node exists in the result
        if len(result.nodes) == 0:
            print(f"No node found with ID {node_id}.")
            return None
        
        # Get the node
        node = result.nodes[0]
        print(f"Node ID: {node.id}")
        print(f"Latitude: {node.lat}")
        print(f"Longitude: {node.lon}")
        print(f"Tags: {node.tags}")
        return node
    except Exception as e:
        print(f"Error searching for node {node_id}: {e}")
        return None

# Example usage
node_id = 123456789  # Replace with the actual node ID
get_node_by_id(node_id)


In [ ]:
import requests

def add_key_to_node(node_id, key, value):
    """
    Add a key-value pair to an existing node in JOSM using the remote control API.
    
    :param node_id: The ID of the node to update.
    :param key: The key to add.
    :param value: The value to add for the key.
    """
    # JOSM remote control base URL
    josm_url = "http://127.0.0.1:8111"

    # Prepare the command for JOSM remote control
    params = {
        "id": node_id,
        "addtags": f"{key}={value}"
    }

    # Send the request to modify the node
    response = requests.get(f"{josm_url}/load_object", params=params)

    if response.status_code == 200:
        print(f"Successfully added key '{key}' with value '{value}' to node {node_id}.")
    else:
        print(f"Failed to add key to node {node_id}. Response: {response.text}")

# Example usage
node_id = 123456789  # Replace with the actual node ID
key = "new_key"      # The key you want to add
value = "new_value"  # The value for the key
add_key_to_node(node_id, key, value)
